### This is a notebook to explore build a chatbot via LangChain Liberary

- Use AzureOpenAI's API
- Create prompt template to set the role for AI
- Use memory component to remember the chat context
- Include an agents to support differnt tasks

In [3]:
import os
import re
from langchain.chat_models import AzureChatOpenAI
from langchain import LLMChain
from langchain.llms import AzureOpenAI
from langchain.agents import load_tools, initialize_agent, AgentType, Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.memory import ConversationBufferWindowMemory, ConversationBufferMemory
from langchain.prompts import BaseChatPromptTemplate
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, HumanMessage
from dotenv import load_dotenv

# load env for llm service
load_dotenv()

True

In [4]:
# create LLM chat model
llm_normal = AzureOpenAI(deployment_name='chatGPTAzure', model='gpt-35-turbo')
llm_chat = AzureChatOpenAI(deployment_name='chatGPTAzure', model='gpt-35-turbo')

llm_normal, llm_chat

(AzureOpenAI(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, client=<class 'openai.api_resources.completion.Completion'>, model_name='gpt-35-turbo', temperature=0.7, max_tokens=256, top_p=1, frequency_penalty=0, presence_penalty=0, n=1, best_of=1, model_kwargs={}, openai_api_key='d2d65893138c46d39ffe1d69f52eda88', openai_api_base='https://qucy-openai-test.openai.azure.com/', openai_organization='', openai_proxy='', batch_size=20, request_timeout=None, logit_bias={}, max_retries=6, streaming=False, allowed_special=set(), disallowed_special='all', tiktoken_model_name=None, deployment_name='chatGPTAzure', openai_api_type='azure', openai_api_version='2023-03-15-preview'),
 AzureChatOpenAI(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-35-turbo', temperature=0.7, model_kwargs={}, openai_api_key='d2d65893138c46d39ffe1d69f52eda88', openai_api_base='h

In [5]:
def hsbc_knowledge_tool(input: str) -> str:
    return """
        Need to open a bank account with HSBC HK? You can apply with us if you:
        - are at least 18 years old
        - meet additional criteria depending on where you live
        - have proof of ID, proof of address

    If customer wants to apply online via mobile app:
        - They need to download the HSBC HK App to open an account online.
        - holding an eligible Hong Kong ID or an overseas passport
        - new to HSBC
    """

def news_query_tool(input: str) -> str:
    return """
        Here are the latest news:
        - 2021-11-01: HSBC Holdings plc 3Q 2021 Earnings Release
        - 2021-10-29: HSBC Holdings plc 3Q 2021 Earnings Release
        - 2021-10-29: HSBC Holdings plc 3Q 2021 Earnings Release
    """

def reject_tool(input: str) -> str:
    return """
    I'm sorry, but as a customer service chatbot for HSBC Hongkong, I am only able to assist with questions related to HSBC Hongkong products and services. 
    Is there anything else related to HSBC Hongkong that I can help you with?
    """

In [6]:
# Define which tools the agent can use to answer user queries
tools = [
    Tool(
        name = "hsbc_knowledge_tool",
        func=hsbc_knowledge_tool,
        description="useful for when you need to answer questions about hsbc related knowledge"
    ),
    Tool(
        name = "news_query_tool",
        func=news_query_tool,
        description="useful for when you need to answer questions about news"
    ),
    Tool(
        name = "reject_tool",
        func=reject_tool,
        description="useful for when you need to answer questions not related to HSBC"
    )
]

In [15]:
# use annotation
from langchain.tools import tool

@tool("hsbc knowledge search tool")
def hsbc_knowledge_tool(input: str) -> str:
    """useful for when you need to answer questions about hsbc related knowledge"""
    return """
        Need to open a bank account with HSBC HK? You can apply with us if you:
        - are at least 18 years old
        - meet additional criteria depending on where you live
        - have proof of ID, proof of address

    If customer wants to apply online via mobile app:
        - They need to download the HSBC HK App to open an account online.
        - holding an eligible Hong Kong ID or an overseas passport
        - new to HSBC
    """

@tool("reject tool", return_direct=True)
def reject_tool(input: str) -> str:
    """ useful for when you need to answer questions not related to HSBC """
    return """
    I'm sorry, but as a customer service chatbot for HSBC Hongkong, I am only able to assist with questions related to HSBC Hongkong products and services. 
    Is there anything else related to HSBC Hongkong that I can help you with?
    """

# construct tools array
tools = [
   hsbc_knowledge_tool,
    Tool(
        name = "news_query_tool",
        func=news_query_tool,
        description="useful for when you need to answer questions about news"
    )
]

In [16]:
# Set up the base template
template = """Complete the objective as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

These were previous tasks you completed:

Begin!

Question: {input}
{agent_scratchpad}"""

In [17]:
# Set up a prompt template
class CustomPromptTemplate(BaseChatPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]
    
    def format_messages(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        formatted = self.template.format(**kwargs)
        return [HumanMessage(content=formatted)]
    
class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [18]:
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    input_variables=["input", "intermediate_steps"]
)

output_parser = CustomOutputParser()

memory = ConversationBufferWindowMemory(k=10)

llm_chain = LLMChain(llm=llm_chat, prompt=prompt, output_parser=output_parser, memory=memory)

tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=tool_names
)
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [19]:
# this is the example proivded by Langchain offically (https://python.langchain.com/docs/modules/agents/how_to/custom_llm_chat_agent) 
# but it will ounter errors and should figure out another way to achevice the same goal
# agent_executor.run("How can I open an account with HSBC HK?")

## =============== Error
#      19 prompt_input_keys = list(set(inputs).difference(memory_variables + ["stop"]))
#      20 if len(prompt_input_keys) != 1:
# ---> 21     raise ValueError(f"One input key expected got {prompt_input_keys}")
#      22 return prompt_input_keys[0]

# ValueError: One input key expected got ['intermediate_steps', 'input']

In [20]:
# We can add prefix for the agent but only works the zero shot react discription type of agent
# for chat conversational react type of agent is not working at all
PREFIX = """
You are a customer service AI for HSBC Hongkong, your primary focus would be to assist customers with questions and issues related to HSBC Hongkong products and services. 
If a customer asks a question that is not related to HSBC Hongkong, politely inform them that I am only able to assist with HSBC Hongkong related questions.
You have access to the following tools:
"""

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

agent_chain = initialize_agent(tools, 
                               llm_chat, 
                               agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, 
                               verbose=True, 
                               memory=memory,
                               agent_kwargs={
                                   'system_message': PREFIX
                               })

In [23]:
agent_chain.agent.llm_chain

LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, prompt=ChatPromptTemplate(input_variables=['input', 'chat_history', 'agent_scratchpad'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template='\nYou are a customer service AI for HSBC Hongkong, your primary focus would be to assist customers with questions and issues related to HSBC Hongkong products and services. \nIf a customer asks a question that is not related to HSBC Hongkong, politely inform them that I am only able to assist with HSBC Hongkong related questions.\nYou have access to the following tools:\n', template_format='f-string', validate_template=True), additional_kwargs={}), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], output_parser=None, partial_variables={}, template='TOOLS\n------\nAss

In [24]:
agent_chain.run(input="hi, i am bob, can you hear me ? ")



> Entering new  chain...
{
    "action": "Final Answer",
    "action_input": "Yes, Bob. I can hear you loud and clear. How may I assist you with HSBC Hongkong related queries today?"
}

> Finished chain.


'Yes, Bob. I can hear you loud and clear. How may I assist you with HSBC Hongkong related queries today?'

In [25]:
agent_chain.run(input="How can I open an HSBC account ?")



> Entering new  chain...
Here is a markdown code snippet of a JSON blob with a single action to take:

```json
{
    "action": "hsbc knowledge search tool",
    "action_input": "How to open an HSBC account"
}
```

Please use the HSBC knowledge search tool and search for "How to open an HSBC account" to find the relevant information.
Observation: 
        Need to open a bank account with HSBC HK? You can apply with us if you:
        - are at least 18 years old
        - meet additional criteria depending on where you live
        - have proof of ID, proof of address

    If customer wants to apply online via mobile app:
        - They need to download the HSBC HK App to open an account online.
        - holding an eligible Hong Kong ID or an overseas passport
        - new to HSBC
    
Thought:Apologies for the confusion earlier. Here is a markdown code snippet of a JSON blob with a single action based on the information obtained from the HSBC knowledge search tool:

```json
{
    "a

'To open an HSBC account, you must be at least 18 years old, meet additional criteria depending on where you live, and have proof of ID and address. If you want to apply online via mobile app, you need to download the HSBC HK App and be holding an eligible Hong Kong ID or an overseas passport. You must also be new to HSBC.'

In [26]:
agent_chain.run(input="Any latest news regarding HSBC ?")



> Entering new  chain...
{
    "action": "news_query_tool",
    "action_input": "HSBC"
}
Observation: 
        Here are the latest news:
        - 2021-11-01: HSBC Holdings plc 3Q 2021 Earnings Release
        - 2021-10-29: HSBC Holdings plc 3Q 2021 Earnings Release
        - 2021-10-29: HSBC Holdings plc 3Q 2021 Earnings Release
    
Thought:{
    "action": "Final Answer",
    "action_input": "Here are some of the latest news regarding HSBC:\n- 2021-11-01: HSBC Holdings plc 3Q 2021 Earnings Release\n- 2021-10-29: HSBC Holdings plc 3Q 2021 Earnings Release\n- 2021-10-29: HSBC Holdings plc 3Q 2021 Earnings Release"
}

> Finished chain.


'Here are some of the latest news regarding HSBC:\n- 2021-11-01: HSBC Holdings plc 3Q 2021 Earnings Release\n- 2021-10-29: HSBC Holdings plc 3Q 2021 Earnings Release\n- 2021-10-29: HSBC Holdings plc 3Q 2021 Earnings Release'

In [27]:
# The problem with CHAT_CONVERSATIONAL_REACT_DESCRIPTION, is that looks like the prompt is not working
# but if you try other type of AgentType will get an error -> can not format the LLM output
# looks like this is the best agent we can have for now !
response = agent_chain.run(input="How can I open a CITI bank account ?")



> Entering new  chain...
{
    "action": "hsbc knowledge search tool",
    "action_input": "Steps to open a CITI bank account"
}
Observation: 
        Need to open a bank account with HSBC HK? You can apply with us if you:
        - are at least 18 years old
        - meet additional criteria depending on where you live
        - have proof of ID, proof of address

    If customer wants to apply online via mobile app:
        - They need to download the HSBC HK App to open an account online.
        - holding an eligible Hong Kong ID or an overseas passport
        - new to HSBC
    
Thought:{
    "action": "Final Answer",
    "action_input": "To open a CITI bank account, you will need to visit a CITI bank branch and bring some identification documents and proof of address. Unfortunately, I am not able to provide specific information about opening a CITI bank account as I am a customer service AI for HSBC Hongkong and I am only able to assist with HSBC Hongkong related questions."
}


In [28]:
response

'To open a CITI bank account, you will need to visit a CITI bank branch and bring some identification documents and proof of address. Unfortunately, I am not able to provide specific information about opening a CITI bank account as I am a customer service AI for HSBC Hongkong and I am only able to assist with HSBC Hongkong related questions.'